In [2]:
!which python

/Users/awlego/Repositories/ml-num-dots/env3.10/bin/python


In [3]:
import ollama
import pandas as pd
import os
from datetime import datetime
import time
from tqdm import tqdm
import itertools
from itertools import product


In [32]:
pkl_name_1_5b = "results/dot_counting_results_deepseek-r1_1.5b_20250227_092228.pkl"
pkl_name_7b = "results/dot_counting_results_deepseek-r1_7b_20250227_092228.pkl"
pkl_name_14b = "results/dot_counting_results_deepseek-r1_14b_20250227_092228.pkl"
pkl_name_32b = "results/dot_counting_results_deepseek-r1_32b_20250227_092228.pkl"

df_1_5b = pd.read_pickle(pkl_name_1_5b)
df_7b = pd.read_pickle(pkl_name_7b)
df_14b = pd.read_pickle(pkl_name_14b)
df_32b = pd.read_pickle(pkl_name_32b)



In [35]:
# Add a 'model' column to each dataframe
df_1_5b['model'] = '1.5b'
df_7b['model'] = '7b'
df_14b['model'] = '14b'
df_32b['model'] = '32b'
# Merge all dataframes into one
df = pd.concat([df_1_5b, df_7b, df_14b, df_32b], ignore_index=True)
model_size_dict = {'1.5b': 1.5, '7b': 7, '14b': 14, '32b': 32}

df.head()


,model,prompt_type,sequence_type,prompt,actual_count,model_answer,extracted_number,correct
0,1.5b,1,1,Do not use code. How many periods/dots '.' are...,1,"<think>\nFirst, I will examine the given seque...",1,True
1,1.5b,1,1,Do not use code. How many periods/dots '.' are...,2,<think>\nI need to determine how many dots (.)...,2,True
2,1.5b,1,1,Do not use code. How many periods/dots '.' are...,3,<think>\nI need to determine how many periods ...,3,True
3,1.5b,1,1,Do not use code. How many periods/dots '.' are...,4,"<think>\nFirst, I will examine the given seque...",3,False
4,1.5b,1,1,Do not use code. How many periods/dots '.' are...,5,<think>\nI need to determine how many periods ...,5,True


In [36]:
df.groupby('model').mean(numeric_only=True)['correct'].sort_index(key=lambda x: x.map(model_size_dict))

model
1.5b               0.28
7b                 0.36
14b                0.52
deepseek-r1:32b    0.57
Name: correct, dtype: float64

better models do better

In [16]:
df.groupby('actual_count').mean(numeric_only=True)['correct']


actual_count
1     1.000000
2     1.000000
3     1.000000
4     0.833333
5     0.666667
6     0.750000
7     0.750000
8     0.666667
9     0.500000
10    0.583333
11    0.166667
12    0.333333
13    0.250000
14    0.166667
15    0.083333
16    0.083333
17    0.000000
18    0.083333
19    0.083333
20    0.250000
21    0.083333
22    0.000000
23    0.083333
24    0.166667
25    0.083333
Name: correct, dtype: float64

generally models get worse as number of dots increases, but there are some exceptions. 12 is better than 11, 20 and 24 also get bumps.

In [29]:
df.groupby('prompt_type').mean(numeric_only=True)['correct']


prompt_type
1    0.406667
2    0.366667
Name: correct, dtype: float64

interesting! the simple prompting is better!

In [30]:
df.groupby('sequence_type').mean(numeric_only=True)['correct']


sequence_type
1    0.333333
2    0.440000
Name: correct, dtype: float64

As expected, tokenizing the sequence with spaces is better.

In [31]:
df.groupby(['sequence_type', 'prompt_type']).mean(numeric_only=True)['correct']


sequence_type  prompt_type
1              1              0.373333
               2              0.293333
2              1              0.440000
               2              0.440000
Name: correct, dtype: float64